Running Calculations from Python
================================

The *tblite* Python package allows running extended tight binding (xTB) calculations directly in Python.
This tutorial demonstrates how to set up and run a single-point calculation using GFN2-xTB.

Installing the Package
----------------------

To start, create a new Python environment using the mamba package manager.
We specify the packages we want to install in our environment file:

```yaml
name: xtb
channels:
- conda-forge
dependencies:
- ipykernel
- tblite-python
- qcelemental
- polars
```

Save the file as *environment.yml* and create the environment by running:

```shell
mamba env create -n xtb -f environment.yml
mamba activate xtb
```

This will create a new environment called *xtb* and install all the necessary packages.
Make sure that *tblite* is available in your Python environment.
You can check this by opening a Python interpreter and importing the package.

In [ ]:
import tblite.interface as tb

tb.library.get_version()

First calculation
-----------------

We will run our first calculation directly from Python.
To specify the input, we will directly initialize the calculation without reading any external files.
For this, we specify our input as Cartesian coordinates in Bohr and atomic numbers for the elements.

In [ ]:
import numpy as np

coordinates = np.array([
    [ 2.02799738646442,  0.09231312124713, -0.14310895950963],
    [ 4.75011007621000,  0.02373496014051, -0.14324124033844],
    [ 6.33434307654413,  2.07098865582721, -0.14235306905930],
    [ 8.72860718071825,  1.38002919517619, -0.14265542523943],
    [ 8.65318821103610, -1.19324866489847, -0.14231527453678],
    [ 6.23857175648671, -2.08353643730276, -0.14218299370797],
    [ 5.63266886875962, -4.69950321056008, -0.13940509630299],
    [ 3.44931709749015, -5.48092386085491, -0.14318454855466],
    [ 7.77508917214346, -6.24427872938674, -0.13107140408805],
    [10.30229550927022, -5.39739796609292, -0.13672168520430],
    [12.07410272485492, -6.91573621641911, -0.13666499342053],
    [10.70038521493902, -2.79078533715849, -0.14148379504141],
    [13.24597858727017, -1.76969072232377, -0.14218299370797],
    [ 7.40891694074004, -8.95905928176407, -0.11636933482904],
    [ 1.38702118184179,  2.05575746325296, -0.14178615122154],
    [ 1.34622199478497, -0.86356704498496,  1.55590600570783],
    [ 1.34624089204623, -0.86133716815647, -1.84340893849267],
    [ 5.65596919189118,  4.00172183859480, -0.14131371969009],
    [14.67430918222276, -3.26230980007732, -0.14344911021228],
    [13.50897177220290, -0.60815166181684,  1.54898960808727],
    [13.50780014200488, -0.60614855212345, -1.83214617078268],
    [ 5.41408424778406, -9.49239668625902, -0.11022772492007],
    [ 8.31919801555568, -9.74947502841788,  1.56539243085954],
    [ 8.31511620712388, -9.76854236502758, -1.79108242206824],
])
elements = np.array([6,7,6,7,6,6,6,8,7,6,8,7,6,6,1,1,1,1,1,1,1,1,1,1])

xtb = tb.Calculator("GFN2-xTB", elements, coordinates)

This input allows us to perform an xTB calculation using the *singlepoint* method, which returns the calculation results to obtain the total energy.

In [ ]:
results = xtb.singlepoint()

results["energy"]

While it is possible to run xTB calculations this way directly in Python, it will become quickly cumbersome if we want to run many calculations at once.
Instead, we want to read our geometry from an input file, for example, an xyz geometry file.

In [ ]:
%%writefile caffeine.xyz
24

C            1.07317        0.04885       -0.07573
N            2.51365        0.01256       -0.07580
C            3.35199        1.09592       -0.07533
N            4.61898        0.73028       -0.07549
C            4.57907       -0.63144       -0.07531
C            3.30131       -1.10256       -0.07524
C            2.98068       -2.48687       -0.07377
O            1.82530       -2.90038       -0.07577
N            4.11440       -3.30433       -0.06936
C            5.45174       -2.85618       -0.07235
O            6.38934       -3.65965       -0.07232
N            5.66240       -1.47682       -0.07487
C            7.00947       -0.93648       -0.07524
C            3.92063       -4.74093       -0.06158
H            0.73398        1.08786       -0.07503
H            0.71239       -0.45698        0.82335
H            0.71240       -0.45580       -0.97549
H            2.99301        2.11762       -0.07478
H            7.76531       -1.72634       -0.07591
H            7.14864       -0.32182        0.81969
H            7.14802       -0.32076       -0.96953
H            2.86501       -5.02316       -0.05833
H            4.40233       -5.15920        0.82837
H            4.40017       -5.16929       -0.94780

Instead of implementing our own xyz file reader, we will use the *qcelemental* package, which already provides this functionality for us.
Fortunately, the qcelemental library stores the geometry already in Bohr, so we do not need to convert the coordinates to input them in our xTB calculation.

In [ ]:
import qcelemental as qcel

molecule = qcel.models.Molecule.from_file("caffeine.xyz")

xtb = tb.Calculator("GFN2-xTB", molecule.atomic_numbers, molecule.geometry)

results = xtb.singlepoint()
results["energy"]

Defining the geometry and elements of the system explicitly or by reading from the xyz file gives the same results.

HOMO-LUMO gap
-------------

The HOMO-LUMO gap refers to the energy difference between the *Highest Occupied Molecular Orbital* (HOMO) and the *Lowest Unoccupied Molecular Orbital* (LUMO) in a molecule. We continue from our previous session and obtain the orbital energies and occupation numbers to find the highest occupied and the lowest unoccupied orbitals.


In [ ]:
orbital_energies = results["orbital-energies"]
orbital_occupations = results["orbital-occupations"]

lumo_index = np.argmax(orbital_occupations)
homo_index = lumo_index - 1
gap = (orbital_energies[lumo_index] - orbital_energies[homo_index]) * qcel.constants.conversion_factor("hartree", "eV")
gap

:::{note}
The gap energy usually reported in *eV* unit. While xTB energies stored in the atomic units (hartree). Thus, we apply the conversion factor obtained from the qcelemental to change the units accordingly.
:::

Computing ionization potential
------------------------------

Now that we can evaluate energies we want to extend the evaluation to other properties with xTB.
Let's compute the vertical ionization potential for caffeine with xTB.
For computing this property we have two options, first we can get the ionization potential directly from our xTB wavefunction by using the energy of the highest occupied orbital. Second we can compute it as the relative energy of the ionization reaction.


In the first option, the HOMO energy approximates the negative vertical ionization potential.

In [ ]:
homo_energy = -orbital_energies[homo_index] * qcel.constants.conversion_factor("hartree", "eV")
homo_energy

In the second option, instead of approximating the ionization potential we can also compute it.
The energy of removing an electron can be expressed by the reaction

$$
\text{Molecule} \rightarrow \text{Molecule}^{+} + e^{-}
$$

This reaction energy is the negative ionization potential.
To compute this energy with xTB we update our calculator by setting the total charge to +1:

In [ ]:
xtb.update(charge=1)
results_ion = xtb.singlepoint()

vertical_ip = (results_ion["energy"] - results["energy"]) * qcel.constants.conversion_factor("hartree", "eV")
vertical_ip

:::{note}
Since we only need to change the total charge in the input, we can update our xTB calculator instead of creating a new one by the *update* method.
:::

We do find quite a difference in the calculated value and the approximated one.
Before we can use the ionization potential computed by xTB we should however correct for the self-interaction error using an empirical determined shift of *4.846 V*.
This shift should be applied for all ionization potentials computed with xTB.
Note the unit of the shift is *Volt (V)* meaning that the correction should be applied for each added or removed electron.

:::{note}
Since xTB is a semiempirical method it makes some approximations which result in a strong self-interaction for a free electron.
This value can be computed exactly from the xTB parameters or determined empirically.{footcite}`neugebauer2020`
:::

Fukui indices from partial charges
----------------------------------

While the molecular ionization potential is a great descriptor for the whole molecule, xTB also provides many properties, which are atom-resolved.
Computing the Fukui index{footcite}`yang1986` provides a simple descriptor for chemical reactivity we can compute from the partial charges according to the following equations: 

$$
f_\text{A}^{(+)} = q_\text{A}^{(0)} - q_\text{A}^{(-)} \qquad
f_\text{A}^{(-)} = q_\text{A}^{(+)} - q_\text{A}^{(0)} \qquad
f_\text{A}^{(0)} = \frac12 \left(q_\text{A}^{(+)} - q_\text{A}^{(-)}\right)
$$

where we have the three Fukui indices computed from the partial charges of the neutral (0), cationic (+) and anionic (-) system.
To perform this calculation with xTB we go back to our computation environment and update our molecule to a negative total charge:

In [ ]:
import polars as pl

xtb.update(charge=-1)
results_neg = xtb.singlepoint()

charges = pl.DataFrame({ 
    "elements": molecule.symbols,
    "q(0)":  results.get("charges"), 
    "q(+)": results_ion.get("charges"),
    "q(-)": results_neg.get("charges"),
})
fukui = charges.select(
    pl.col("elements"),
    (pl.col("q(0)") - pl.col("q(-)")).alias("f(+)"),
    (pl.col("q(+)") - pl.col("q(0)")).alias("f(-)"),
    ((pl.col("q(+)") - pl.col("q(-)")) / 2).alias("f(0)"),
)
fukui

Localizing frontier orbitals
----------------------------

The xTB methods are using a minimal basis set, which allows to localize many orbital derived properties by Mulliken population analysis.
The partial charges we used previously in the Fukui index calculation are one example and are already computed in xTB and therefore readily available.
Here we will localize the frontier orbitals to the atomic centers using integrals and matrices we can compute in xTB.

For this we define the population of the occupied and virtual orbitals and compute their Mulliken population:

$$
   P^\text{occ/vir}_{\text{A}i/a} = 2 \sum_{\kappa\in\text{A}} \sum_\lambda^{N_\text{ao}}  C^\text{occ/vir}_{\kappa i/a} \cdot S_{\kappa\lambda} \cdot C^\text{occ/vir}_{\lambda i/a}
$$

here *C* are the orbital coefficients, *S* the overlap matrix in the atomic orbital basis, A is the atom index, *κ*/*λ* are the atomic orbitals, $N_\text{ao}$ is the number of the atomic orbitals, *i* is the occupied orbital index and *a* is the virtual orbital index.

To compute the local response function *Λ* we compute the difference between the occupied and virtual orbitals and localize them on each of the atoms using the Mulliken population matrices defined before

$$
   \Lambda_\text{A} = \sum_{a,i}\frac{P^\text{occ}_{\text{A}i} P^\text{vir}_{\text{A}a}}{(\varepsilon^\text{vir}_a - \varepsilon^\text{occ}_i)^2 + \eta^2}
$$

where *ε* are the orbital energies, *η* is a parameter for regularization here set to 0.5eV.

The Fermi level *μ* is computed by taking the average of the occupied and virtual orbital energies divided by the local response function.

$$
   \mu_{\text{A}} = \frac1{2\Lambda_\text{A}}\sum_{a,i}\frac{P^\text{occ}_{\text{A}i} P^\text{vir}_{\text{A}a} (\varepsilon^\text{vir}_a + \varepsilon^\text{occ}_i)}{(\varepsilon^\text{vir}_a - \varepsilon^\text{occ}_i)^2 + \eta^2}
$$

The localized gap *δ* finally is computed from the localized difference of the orbital energies

$$
   \delta_{\text{A}} = \Lambda_\text{A} \cdot \left(\sum_{a,i}\frac{P^\text{occ}_{\text{A}i} P^\text{vir}_{\text{A}a}}{(\varepsilon^\text{vir}_a - \varepsilon^\text{occ}_i)^3 + \eta^3}\right)^{-1} - \eta
$$

We can implement these equations and compute the frontier orbitals based on the orbital eigenvalues, occupation numbers, orbital coefficients and overlap matrix.

In [ ]:
def localize_frontier_orbitals(
    eigenvalues: np.ndarray,  # Orbital energies in eV (nao,)
    occupations: np.ndarray,  # Orbital occupations (nao,)
    coeff: np.ndarray,  # Orbital coefficients (nao, nao)
    overlap: np.ndarray,  # Overlap matrix (nao, nao)
    orbital_to_atom: np.ndarray,  # Orbital to atom mapping (nao,)
    eta: float = 0.5,  # eV
):
    eps = 1e-12

    nao = orbital_to_atom.size
    atom_projection = np.zeros((np.max(orbital_to_atom) + 1, nao))
    atom_projection[orbital_to_atom, np.arange(nao)] = 1

    occ_indices = np.where(occupations > eps)[0]  # (n_occ,)
    vir_indices = np.where((1.0 - occupations) > eps)[0]  # (n_vir)

    pop = 2 * (atom_projection @ (coeff * (overlap @ coeff)))  # (nat, nao)
    pop_vir = pop[:, vir_indices]  # (nat, n_vir)
    pop_occ = pop[:, occ_indices]  # (nat, n_occ)

    eps_diff = eigenvalues[vir_indices, None] - eigenvalues[None, occ_indices]  # (n_vir, n_occ)
    div_eps2 = 1.0 / (eps_diff**2 + eta**2)  # (n_vir, n_occ)
    div_eps3 = 1.0 / (eps_diff**3 + eta**3)  # (n_vir, n_occ)
    eps_sum = (eigenvalues[vir_indices, None] + eigenvalues[None, occ_indices]) * div_eps2  # (n_vir, n_occ)

    response = np.einsum("xi,xa,ai->x", pop_occ, pop_vir, div_eps2)  # (nat,)
    mask = response > eps

    chemical_potential = np.einsum("xi,xa,ai->x", pop_occ, pop_vir, eps_sum) / 2  # (nat,)
    chemical_potential = np.where(mask, chemical_potential / response, 0.0)

    local_gap = np.einsum("xi,xa,ai->x", pop_occ, pop_vir, div_eps3)  # (nat,)
    local_gap = np.where(mask, 1.0 / (local_gap / response + eps) - eta, 0.0)

    return chemical_potential, local_gap

:::{note}
The frontier orbital computation is also implemented in the xTB calculator and can be used without any integral calculation in Python.
Here we reimplement a simplified version of this computation as an example for localization and population analysis possible in xTB.
:::

Now we can compute these frontier orbitals with xTB.
First, we need to explicitly ask to save the integrals during our calculation using the *set* method of the xTB calculator to change the *save-integrals* property.
Another important point to consider is the charge of the system. Since our example is the caffeine molecule, the charge is updated to zero.


In [ ]:
xtb.set("save-integrals", True)
xtb.update(charge=0)
results = xtb.singlepoint(results)

:::{note}
Notice only two iterations are needed for the single point since the calculations is restarted from the previous *results*.
:::


To obtain the required properties we can now query the results we got from our calculation with the updated xTB calculator.
Some properties like the orbital to shell and the shell to atom mapping we can directly obtain from the calculator instead of the calculation results.

In [ ]:
orbital_map = xtb.get("orbital-map")
shell_map = xtb.get("shell-map")
orbital_to_atom = shell_map[orbital_map]

chemical_potential, local_gap = localize_frontier_orbitals(
   eigenvalues=results["orbital-energies"] * qcel.constants.conversion_factor("hartree", "eV"),
   occupations=results["orbital-occupations"],
   coeff=results["orbital-coefficients"],
   overlap=results["overlap-matrix"],
   orbital_to_atom=orbital_to_atom,
)

frontier_orbitals = pl.DataFrame({
    "element": molecule.symbols,
    "Fermi level [eV]": chemical_potential,
    "local gap [eV]": local_gap,
    "highest occ. AO [eV]": chemical_potential - local_gap / 2,
    "lowest unocc. AO [eV]": chemical_potential + local_gap / 2,
})
frontier_orbitals

Summary
-------

In this tutorial, we demonstrated how to set up and run singlepoint calculations using the *tblite* Python package.
We explored two methods for defining molecular geometry: explicitly in Python and by reading from an xyz file using the *qcelemental* package.
Furthermore, we computed properties based on our xTB calculation results, including

* the HOMO-LUMO gap
* the vertical ionization potential
* Fukui indices using partial charges from calculations with different total charge
* localized atomic HOMO-LUMO gaps using integrals computed by xTB

With this introduction, you can perform extended tight binding (xTB) calculations and derive properties from the calculation results.

Literature
----------

:::{footbibliography}
:::